In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('2015.csv')

In [3]:
''' 
SEX ... home o dona
NUMADULT ... Numero adults vivin junts / telef fixe
HHADULT ... Numero adults vivin junts  / telef mobil
GENHLTH ... Opinio personal de la salud de cadascu
CHECKUP1 ... Quant temps desde la ultima revisió?
BPHIGH4 ... Pressió sanguinea alta, diagnosticat?
BPMEDS ... Mediques la alta pressió?
TOLDHI2 ... Colesterol alt?
CVDINFR4 ... Has tingut mai un atac de cor?
ASTHMA3 ... Tens asma?
ADDEPEV2 ... Mai tingut depressio?
CHILDREN ... Numero de nens a casa menors
SMOKE100 ... fumes?
MARITAL ... casat, divorciat, viudo, separat, parella, solter
MENTHLTH ... Numero de dies en que vas tenir mala salut mental el passat mes
'''
df = df[['SEX', 'NUMADULT', 'HHADULT', 'GENHLTH', 'MENTHLTH',
        'CHECKUP1', 'BPHIGH4', 'BPMEDS', 'TOLDHI2', 'CVDINFR4', 'ASTHMA3', 'ADDEPEV2',
        'MARITAL', 'CHILDREN', 'SMOKE100']].copy()

In [4]:
# Juntem les dos columnes de adults vivint junts, de mobil i de telefon
df['NUMADULT'].fillna(df['HHADULT'], inplace=True)

# Carreguem CADULT que era major de edat o no


# Eliminem la columna secundaria
del df['HHADULT']

# Eliminem les files que no contenen informació de la salut en el moment
df.dropna(subset=['GENHLTH'], inplace=True)

# Eliminem la informacio de ultima revisio que sigui erronia
df.drop(df[df['CHECKUP1'] > 4].index)
df.dropna(subset=['CHECKUP1'], inplace=True)

# Nomes ens quedem amb els valors 1 i 0
df.drop(df[(df['BPHIGH4'] != 1) & (df['BPHIGH4'] != 3)].index)
df.dropna(subset=['BPHIGH4'], inplace=True)
df.BPHIGH4.replace(df[df['BPHIGH4'] == 3].index, 0, inplace=True)

# La gent que no ha contestat si es pren medicament, intuirem que no
df.BPMEDS.replace(df[df['BPMEDS'] != 1].index, 0, inplace=True)
df.BPMEDS.fillna(value=0, inplace=True)

# Eliminant info de colesterol
df.drop(df[(df['TOLDHI2'] != 1) & (df['TOLDHI2'] != 2)].index)
df.dropna(subset=['TOLDHI2'], inplace=True)
df.TOLDHI2.replace(df[df['TOLDHI2'] == 2].index, 0, inplace=True)


# El mateix amb atacs de cor
df.drop(df[(df['CVDINFR4'] != 1) & (df['CVDINFR4'] != 2)].index)
df.dropna(subset=['CVDINFR4'], inplace=True)
df.CVDINFR4.replace(df[df['CVDINFR4'] == 2].index, 0, inplace=True)

# El mateix amb asma
df.drop(df[(df['ASTHMA3'] != 1) & (df['ASTHMA3'] != 2)].index)
df.dropna(subset=['ASTHMA3'], inplace=True)
df.ASTHMA3.replace(df[df['ASTHMA3'] == 2].index, 0, inplace=True)

# El mateix amb depressio
df.drop(df[(df['ADDEPEV2'] != 1) & (df['ADDEPEV2'] != 2)].index)
df.dropna(subset=['ADDEPEV2'], inplace=True)
df.ADDEPEV2.replace(df[df['ADDEPEV2'] == 2].index, 0, inplace=True)

# Eliminem les columnes faltants de children
df.drop(df[df['CHILDREN'] > 20].index)
df.dropna(subset=['CHILDREN'], inplace=True)

# Eliminem les de SMOKE
df.drop(df[df['SMOKE100'] > 2].index)
df.dropna(subset=['SMOKE100'], inplace=True)

# Creem la variable objectiu utilitzant MENTHLTH
df.MENTHLTH.replace(df[df['MENTHLTH'] > 30].index, 0, inplace=True)
df.MENTHLTH.replace(df[df['MENTHLTH'] <= 30].index, 1, inplace=True)
df['y'] = df['MENTHLTH'].copy()
del df['MENTHLTH']

In [7]:
df[df.SMOKE100>2]

,SEX,NUMADULT,GENHLTH,CHECKUP1,BPHIGH4,BPMEDS,TOLDHI2,CVDINFR4,ASTHMA3,ADDEPEV2,MARITAL,CHILDREN,SMOKE100,y
287,1.0,1.0,1.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0,3.0,88.0,7.0,1.0
491,2.0,2.0,2.0,1.0,3.0,0.0,2.0,0.0,0.0,0.0,1.0,88.0,7.0,1.0
656,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,88.0,9.0,1.0
747,1.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,88.0,7.0,1.0
1173,2.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,88.0,9.0,1.0
1217,2.0,5.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,88.0,7.0,1.0
1464,1.0,2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,88.0,9.0,1.0
1785,1.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,88.0,7.0,1.0
1919,2.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,88.0,7.0,1.0
2214,2.0,1.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,88.0,7.0,1.0
